# QUANTUM ESPRESSO Example App

**Author: Aliaksandr Yakutovich (LSMO/THEOS, EPFL)**

This apps allows to compute energy, band structure or to optimize the geometry of a material with a minimal set of input parameters.

It is powered by:
- [Quantum ESPRESSO](https://www.quantum-espresso.org/) as the quantum engine
- [AiiDA](http://www.aiida.net) as the automation platform
- [AiiDA-QUANTUMESPRESSO](https://github.com/aiidateam/aiida-quantumespresso) plugin including custom-made workflowsfor AiiDA to manage the selection of parameters, the error handling, etc.

### Example steps to run:
1. SCF
1. Relaxation
1. Band structure calculation

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import os
import sys
from time import sleep 

orig = sys.stderr
sys.stderr = sys.stdout # redirect the stderr to stdout
import ipywidgets as ipw
from aiidalab_widgets_base import aiidalab_display, CodeDropdown, CodQueryWidget, ExportButton
from aiidalab_widgets_base import ProgressBar,StructureUploadWidget, SubmitButtonWidget

from aiida.orm import Dict, Float, KpointsData, load_node, Str
from aiida.engine import submit
from aiida.plugins import WorkflowFactory

In [ ]:
arrow_down = ipw.HTML("""<hr>
                         <br />
                         <center>
                         <i class="fa fa-arrow-down" style="color:#B0B0B0;font-size:12em;"></i>
                         </center>
                        """)
hr = ipw.HTML('<hr>')
run_btn = ipw.Button(description='Submit calculation')
code_group = CodeDropdown(input_plugin='quantumespresso.pw', text="Select code")
number_of_nodes = ipw.IntText(value=1,
                              step=1,description = "that will be run on",
                              disabled=False,
                              layout=ipw.Layout(width="180px"),
                              style={"description_width":"120px"},)
cpus_per_node = ipw.IntText(
    value=1,
    step=1,
    description = "",
    disabled=False,
    layout=ipw.Layout(width="50px"),
    style={"description_width":"0px"},
)
structure_widget = StructureUploadWidget(
    examples=[
        ('Silicon', 'miscellaneous/structures/Si.xyz'),
        ('GaAs','miscellaneous/structures/GaAs.xyz')
    ],
    data_importers=[("COD", CodQueryWidget())],
    storable=False,
    node_class='StructureData')

def change_submit_calc_visibility(c):
    submit_out.layout.visibility = 'visible' if c['new'] else 'hidden'
structure_widget.observe(change_submit_calc_visibility, names=['has_structure'])

In [ ]:
def setup_calc():
    if code_group.selected_code is None:
        print ("Please select a code")
        return None
    
    if 'optimized_structure' not in globals() and not structure_widget.structure_node:
        print ("Please select a structure")
        return None 
    builder = WorkflowFactory(run_type.value).get_builder()
    params = Dict(dict={
            'SYSTEM': {
                'ecutwfc': 50.,
                'ecutrho': 200.,
            },
        })
    options = {
                'max_wallclock_seconds': 3600*2,
                'resources':{
                    'num_machines': number_of_nodes.value,
                    'num_mpiprocs_per_machine': cpus_per_node.value,
                }
            }
    if 'optimized_structure' in globals():
        structure = optimized_structure
    else:
        structure = structure_widget.structure_node

    if run_type.value == 'quantumespresso.pw.base':
        builder.pw.code = code_group.selected_code
        builder.pw.parameters = params
        builder.pw.metadata.options = options
        builder.kpoints_distance = Float(0.8)
        builder.pseudo_family = Str(pseudo_family.value)
        builder.pw.structure = structure

    elif run_type.value == 'quantumespresso.pw.relax':
        builder.base.pw.code = code_group.selected_code
        builder.base.pw.parameters = params
        builder.base.pw.metadata.options = options
        builder.base.kpoints_distance = Float(0.8)
        builder.base.pseudo_family = Str(pseudo_family.value)
        builder.structure = structure

    elif run_type.value == 'quantumespresso.pw.bands':
        builder.scf.pw.code = code_group.selected_code
        builder.bands.pw.code = code_group.selected_code
        builder.scf.pw.parameters = params
        builder.bands.pw.parameters = params
        builder.scf.pw.metadata.options = options
        builder.bands.pw.metadata.options = options
        builder.scf.kpoints_distance = Float(0.8)
        builder.scf.pseudo_family = Str(pseudo_family.value)
        builder.bands.pseudo_family = Str(pseudo_family.value)
        builder.structure = structure

    return builder

In [ ]:
def get_running_calculation(process):
    from aiida.orm import CalcJobNode, WorkChainNode
    while issubclass(type(process), WorkChainNode):
        try:
            for label, link in process.get_outputs_dict().items():
                if label.startswith('CALL_') and not link.is_sealed:
                    process = link
                    break
            else:
                return None
        except:
            return None

    if issubclass(type(process), CalcJobNode):
        return process
    else:
        return None

In [ ]:
def submit_process(b):
    global process_outputs
    output_text = ipw.Textarea(
        value="",
        placeholder="Calculation output will appear here",
        description='Calculation output:',
        layout={'width':"900px", 'height':'300px'},
        disabled=False,
        style = {'description_width': 'initial'}
    )
    builder = setup_calc()
    if builder:
        process = submit(builder)
        process_node = load_node(process.id)
        export_btn = ExportButton(process=process)
        process_bar = ProgressBar(process=process_node)
        display(export_btn, output_text, process_bar)
        previous_calc_pk = 0
        while not process_node.is_sealed:
            calc = get_running_calculation(process_node)
            try:
                if calc.pk != previous_calc_pk:
                    lines = []
                    previous_calc_pk = calc.pk
                    output_text.value = ''
                remote_path = calc.get_outputs_dict()['remote_folder'].get_remote_path()
                with open(os.path.join(remote_path, 'aiida.out')) as fobj:
                    lines += fobj.readlines()[len(lines):-1]
                    output_text.value = ''.join(lines)
            except:
                pass
            process_bar.update_state()
            sleep(0.1)
        process_bar.update_state() # updating the state for the last time to be 100% sure
        output_text.layout={'visibility':'hidden'}
        if 'output_structure' in process_node.outputs:
            global optimized_structure
            optimized_structure = process_node.outputs.output_structure
        show_results(process_node.outputs)

In [ ]:
def process_settings():
    global run_type, pseudo_family
    run_type = ipw.ToggleButtons(
        options=[
            ('scf', 'quantumespresso.pw.base'),
            ('relax', 'quantumespresso.pw.relax'),
            ('bands', 'quantumespresso.pw.bands'),
        ],
        description='Calculation type:',
        style = {'description_width': 'initial'},
    )
    pseudo_family = ipw.ToggleButtons(
        options = {
            'SSSP efficiency': 'SSSP_efficiency_v1.0',
            'SSSP accuracy': 'SSSP_precision_v1.0',
        },
        description='Pseudopotential family:',
        style = {'description_width': 'initial'},
    )
    
    display(arrow_down,
            ipw.HBox([code_group, number_of_nodes, ipw.HTML("node(s)"), cpus_per_node, ipw.HTML("CPU each")]),
            run_type,
            pseudo_family,
            run_btn)

In [ ]:
def show_results(outputs):
    for key in ['output_parameters', 'output_structure', 'band_structure', 'retrieved']:
        if key in outputs:
            aiidalab_display(outputs[key])
    process_settings()

In [ ]:
submit_out = ipw.Output(layout={'visibility':'hidden'})
with submit_out:
    run_btn.on_click(submit_process)
    process_settings()
display(structure_widget, submit_out)